In [2]:
import os
import random
from pathlib import Path

# Define paths relative to your notebook location
base_path = Path("Blades Damages Images + Labels/NordTank586x371")
images_dir = base_path / "images"
labels_dir = base_path / "labels"

def analyze_dataset():
    """Analyze the dataset and categorize files by damage presence"""
    damage_files = []
    non_damage_files = []
    
    # Get all label files
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    print(f"Found {len(label_files)} label files")
    
    for label_file in label_files:
        label_path = labels_dir / label_file
        
        # Check if corresponding image exists (try both .png and .jpg)
        img_name_png = label_file.replace('.txt', '.png')
        img_name_jpg = label_file.replace('.txt', '.jpg')
        
        img_path = None
        if (images_dir / img_name_png).exists():
            img_path = images_dir / img_name_png
            img_name = img_name_png
        elif (images_dir / img_name_jpg).exists():
            img_path = images_dir / img_name_jpg
            img_name = img_name_jpg
        
        if img_path is None:
            print(f"No image found for {label_file}")
            continue
            
        # Read label file and check for class 0 (damage)
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:  # Empty file = no annotations = non-damage
            non_damage_files.append((img_name, label_file))
            continue
            
        # Check if any annotation is class 0 (damage)
        has_damage = False
        for line in lines:
            if line.strip():  # Skip empty lines
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            damage_files.append((img_name, label_file))
        else:
            non_damage_files.append((img_name, label_file))
    
    return damage_files, non_damage_files

# Test if paths exist first
print(f"Images directory exists: {images_dir.exists()}")
print(f"Labels directory exists: {labels_dir.exists()}")

if images_dir.exists() and labels_dir.exists():
    # Analyze the dataset
    damage_pairs, non_damage_pairs = analyze_dataset()
    
    print(f"Found {len(damage_pairs)} images with damage")
    print(f"Found {len(non_damage_pairs)} images without damage")
else:
    print("Directory paths are incorrect!")

Images directory exists: True
Labels directory exists: True
Found 2996 label files
No image found for labels.txt
Found 563 images with damage
Found 2432 images without damage


In [3]:
# Sample the desired amounts
max_damage = min(100, len(damage_pairs))
max_non_damage = min(400, len(non_damage_pairs))

# Create final selection
selected_damage = random.sample(damage_pairs, max_damage)
selected_non_damage = random.sample(non_damage_pairs, max_non_damage)

# Combine all selected pairs
all_selected_pairs = selected_damage + selected_non_damage
random.shuffle(all_selected_pairs)  # Shuffle for training

# Extract separate lists
selected_images = [pair[0] for pair in all_selected_pairs]
selected_labels = [pair[1] for pair in all_selected_pairs]

print(f"Final training selection:")
print(f"- {len(selected_damage)} damage examples")
print(f"- {len(selected_non_damage)} non-damage examples") 
print(f"- {len(all_selected_pairs)} total examples")

# Show some examples
print(f"\nFirst 5 selected files:")
for i in range(min(5, len(selected_images))):
    print(f"  {selected_images[i]} <-> {selected_labels[i]}")

# Create train/validation split
from sklearn.model_selection import train_test_split
train_pairs, val_pairs = train_test_split(all_selected_pairs, test_size=0.2, random_state=42)

print(f"\nTrain/Validation split:")
print(f"- Training: {len(train_pairs)} examples")
print(f"- Validation: {len(val_pairs)} examples")

Final training selection:
- 100 damage examples
- 400 non-damage examples
- 500 total examples

First 5 selected files:
  DJI_0785_02_06.png <-> DJI_0785_02_06.txt
  DJI_0583_05_05.png <-> DJI_0583_05_05.txt
  DJI_0770_02_04.png <-> DJI_0770_02_04.txt
  DJI_0022_04_03.png <-> DJI_0022_04_03.txt
  DJI_0367_02_08.png <-> DJI_0367_02_08.txt

Train/Validation split:
- Training: 400 examples
- Validation: 100 examples


In [4]:
import os
import random
from pathlib import Path

# Step 1: Recreate base path
base_path = Path("Blades Damages Images + Labels/NordTank586x371")
images_dir = base_path / "images"
labels_dir = base_path / "labels"

# Step 2: Recreate data selection (simplified version)
def analyze_dataset():
    damage_files = []
    non_damage_files = []
    
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        label_path = labels_dir / label_file
        
        # Check if corresponding image exists
        img_name_png = label_file.replace('.txt', '.png')
        if not (images_dir / img_name_png).exists():
            continue
            
        # Read label file and check for class 0 (damage)
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            non_damage_files.append((img_name_png, label_file))
            continue
            
        has_damage = False
        for line in lines:
            if line.strip():
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            damage_files.append((img_name_png, label_file))
        else:
            non_damage_files.append((img_name_png, label_file))
    
    return damage_files, non_damage_files

# Step 3: Recreate selection
damage_pairs, non_damage_pairs = analyze_dataset()
print(f"Found {len(damage_pairs)} damage images")
print(f"Found {len(non_damage_pairs)} non-damage images")

# Sample the desired amounts
max_damage = min(100, len(damage_pairs))
max_non_damage = min(400, len(non_damage_pairs))

selected_damage = random.sample(damage_pairs, max_damage)
selected_non_damage = random.sample(non_damage_pairs, max_non_damage)

# Combine and split
all_selected_pairs = selected_damage + selected_non_damage
random.shuffle(all_selected_pairs)

# Create train/val split
split_point = int(0.8 * len(all_selected_pairs))
train_pairs = all_selected_pairs[:split_point]
val_pairs = all_selected_pairs[split_point:]

print(f"Training: {len(train_pairs)} examples")
print(f"Validation: {len(val_pairs)} examples")

Found 563 damage images
Found 2432 non-damage images
Training: 400 examples
Validation: 100 examples


In [5]:
# Step 4: Create corrected train and val lists with full relative paths
train_files = [pair[0] for pair in train_pairs]
val_files = [pair[0] for pair in val_pairs]

# Write train list with correct paths
with open('train.txt', 'w') as f:
    for img_name in train_files:
        f.write(f"{base_path}/images/{img_name}\n")

# Write val list with correct paths
with open('val.txt', 'w') as f:
    for img_name in val_files:
        f.write(f"{base_path}/images/{img_name}\n")

print(f"Updated train.txt with {len(train_files)} images")
print(f"Updated val.txt with {len(val_files)} images")

# Update the dataset.yaml to use the corrected file lists
dataset_config = """
train: train.txt
val: val.txt

nc: 2
names: ['damage', 'dirty']
"""

with open('dataset.yaml', 'w') as f:
    f.write(dataset_config)

print("Updated dataset.yaml with correct paths")

Updated train.txt with 400 images
Updated val.txt with 100 images
Updated dataset.yaml with correct paths


In [6]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='wind_turbine_damage'
)

Ultralytics 8.3.202  Python-3.12.7 torch-2.8.0+cpu CPU (Intel Core i5-1035G4 1.10GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=wind_turbine_damage4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=

In [7]:
#Test Model With Unused Images
import os
import random
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

# Load your trained model
model = YOLO('runs/detect/wind_turbine_damage4/weights/best.pt')

# Set up paths
base_path = Path("Blades Damages Images + Labels/NordTank586x371")
images_dir = base_path / "images"
labels_dir = base_path / "labels"

# Get all available images and their classes
def get_unused_images():
    """Find images that weren't used in training"""
    damage_files = []
    dirty_files = []
    
    # Read the training file list to exclude used images
    with open('train.txt', 'r') as f:
        train_images = set(line.strip().split('/')[-1] for line in f)
    with open('val.txt', 'r') as f:
        val_images = set(line.strip().split('/')[-1] for line in f)
    
    used_images = train_images.union(val_images)
    
    # Categorize unused images
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        img_name = label_file.replace('.txt', '.png')
        
        # Skip if this image was used in training/validation
        if img_name in used_images:
            continue
            
        # Check if image exists
        if not (images_dir / img_name).exists():
            continue
            
        # Check class
        label_path = labels_dir / label_file
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            dirty_files.append(img_name)
            continue
            
        has_damage = False
        for line in lines:
            if line.strip():
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            damage_files.append(img_name)
        else:
            dirty_files.append(img_name)
    
    return damage_files, dirty_files

# Get unused images
unused_damage, unused_dirty = get_unused_images()
print(f"Available unused images: {len(unused_damage)} damage, {len(unused_dirty)} dirty")

# Sample test set
test_damage = random.sample(unused_damage, min(20, len(unused_damage)))
test_dirty = random.sample(unused_dirty, min(80, len(unused_dirty)))

print(f"Testing on: {len(test_damage)} damage images, {len(test_dirty)} dirty images")

# Test the model
def test_model_on_images(image_list, true_class_name):
    """Test model on a list of images and return results"""
    results = []
    
    for img_name in image_list:
        img_path = images_dir / img_name
        
        # Run prediction
        pred_results = model(str(img_path))
        
        # Extract predictions
        detections = []
        if len(pred_results[0].boxes) > 0:
            boxes = pred_results[0].boxes
            for box in boxes:
                class_id = int(box.cls[0].item())
                confidence = float(box.conf[0].item())
                class_name = "damage" if class_id == 0 else "dirty"
                detections.append((class_name, confidence))
        
        results.append({
            'image': img_name,
            'true_class': true_class_name,
            'detections': detections
        })
    
    return results

# Run tests
print("\nTesting damage images...")
damage_results = test_model_on_images(test_damage, 'damage')

print("Testing dirty images...")
dirty_results = test_model_on_images(test_dirty, 'dirty')

# Analyze results
def analyze_results(results, target_class):
    """Analyze prediction accuracy"""
    correct = 0
    total = len(results)
    
    print(f"\n--- {target_class.upper()} IMAGES ANALYSIS ---")
    
    for result in results:
        img_name = result['image']
        detections = result['detections']
        
        # Check if any detection matches the true class
        predicted_classes = [det[0] for det in detections]
        has_correct_prediction = target_class in predicted_classes
        
        if has_correct_prediction:
            correct += 1
            
        print(f"{img_name}: {detections if detections else 'No detections'}")
    
    accuracy = correct / total if total > 0 else 0
    print(f"\nAccuracy: {correct}/{total} = {accuracy:.3f}")
    return accuracy

# Analyze results
damage_accuracy = analyze_results(damage_results, 'damage')
dirty_accuracy = analyze_results(dirty_results, 'dirty')

print(f"\n=== OVERALL RESULTS ===")
print(f"Damage detection accuracy: {damage_accuracy:.3f}")
print(f"Dirty detection accuracy: {dirty_accuracy:.3f}")
print(f"Overall accuracy: {(damage_accuracy * len(test_damage) + dirty_accuracy * len(test_dirty)) / (len(test_damage) + len(test_dirty)):.3f}")

# Optional: Show some example predictions visually
print(f"\nModel saved results to: runs/detect/wind_turbine_damage4/")
print("Run model.predict() on individual images to see visual results")

Available unused images: 463 damage, 2032 dirty
Testing on: 20 damage images, 80 dirty images

Testing damage images...

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0618_01_07.png: 416x640 1 damage, 1 dirty, 269.9ms
Speed: 17.5ms preprocess, 269.9ms inference, 10.6ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0623_02_08.png: 416x640 1 damage, 114.2ms
Speed: 6.3ms preprocess, 114.2ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0631_07_04.png: 416x640 1 damage, 112.1ms
Speed: 4.6ms preprocess, 112.1ms inference, 1.7ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0748_04_07.png: 416x640 1 damage, 7 dirtys, 112.4ms
Speed

In [ ]:
import os
import random
from pathlib import Path
from ultralytics import YOLO

#Next version with more data and false negative focused penalty

# Step 1: Expand data selection to 1500 images
base_path = Path("Blades Damages Images + Labels/NordTank586x371")
images_dir = base_path / "images"
labels_dir = base_path / "labels"

def get_all_categorized_images():
    """Get all available images categorized by damage type"""
    damage_files = []
    non_damage_files = []
    
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        label_path = labels_dir / label_file
        img_name_png = label_file.replace('.txt', '.png')
        
        if not (images_dir / img_name_png).exists():
            continue
            
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            non_damage_files.append((img_name_png, label_file))
            continue
            
        has_damage = False
        for line in lines:
            if line.strip():
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            damage_files.append((img_name_png, label_file))
        else:
            non_damage_files.append((img_name_png, label_file))
    
    return damage_files, non_damage_files

# Get all available data
all_damage, all_non_damage = get_all_categorized_images()
print(f"Available: {len(all_damage)} damage, {len(all_non_damage)} non-damage")

# Sample expanded dataset
max_damage = min(300, len(all_damage))
max_non_damage = min(1200, len(all_non_damage))

selected_damage = random.sample(all_damage, max_damage)
selected_non_damage = random.sample(all_non_damage, max_non_damage)

# Create train/val split (80/20)
all_selected = selected_damage + selected_non_damage
random.shuffle(all_selected)

split_point = int(0.8 * len(all_selected))
train_pairs_extended = all_selected[:split_point]
val_pairs_extended = all_selected[split_point:]

print(f"Extended dataset: {len(train_pairs_extended)} train, {len(val_pairs_extended)} val")

# Step 2: Create new file lists
train_files_ext = [pair[0] for pair in train_pairs_extended]
val_files_ext = [pair[0] for pair in val_pairs_extended]

with open('train_extended.txt', 'w') as f:
    for img_name in train_files_ext:
        f.write(f"{base_path}/images/{img_name}\n")

with open('val_extended.txt', 'w') as f:
    for img_name in val_files_ext:
        f.write(f"{base_path}/images/{img_name}\n")

# Step 3: Create dataset config
dataset_config_extended = """
train: train_extended.txt
val: val_extended.txt

nc: 2
names: ['damage', 'dirty']
"""

with open('dataset_extended.yaml', 'w') as f:
    f.write(dataset_config_extended)

print("Created extended dataset files")

# Step 4: Load your existing best model and continue training
model = YOLO('runs/detect/wind_turbine_damage4/weights/best.pt')

results = model.train(
    data='dataset_extended.yaml',
    epochs=80,            # Fewer epochs since we're fine-tuning
    imgsz=640,
    batch=16,
    
    # Damage-focused loss weighting
    cls=3.0,              # Higher classification loss weight
    box=1.5,              # Slightly higher box regression loss
    
    # Fine-tuning parameters
    lr0=0.001,            # Lower learning rate for fine-tuning
    warmup_epochs=5,      # Warm up period
    patience=20,          # Early stopping patience
    
    name='damage_sensitive_finetuned',
    
    # This ensures we continue from your existing weights
    resume=False          # Don't resume training, but start from pretrained weights
)

print("Fine-tuning completed! Check results in runs/detect/damage_sensitive_finetuned/")

Available: 563 damage, 2432 non-damage
Extended dataset: 1200 train, 300 val
Created extended dataset files
Ultralytics 8.3.202  Python-3.12.7 torch-2.8.0+cpu CPU (Intel Core i5-1035G4 1.10GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=1.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=3.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_extended.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/wind_turbine_damage4/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, n

In [1]:
from ultralytics import YOLO

# Resume training from the last checkpoint
model = YOLO('runs/detect/damage_sensitive_finetuned/weights/last.pt')

results = model.train(
    resume=True  # This will resume from the last saved checkpoint
)

Ultralytics 8.3.202  Python-3.12.7 torch-2.8.0+cpu CPU (Intel Core i5-1035G4 1.10GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=1.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=3.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_extended.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs\detect\damage_sensitive_finetuned\weights\last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=damage_sensitive_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap

In [2]:
import os
import random
from pathlib import Path
from ultralytics import YOLO

# Load your new fine-tuned model
model = YOLO('runs/detect/damage_sensitive_finetuned/weights/best.pt')

# Set up paths
base_path = Path("Blades Damages Images + Labels/NordTank586x371")
images_dir = base_path / "images"
labels_dir = base_path / "labels"

def get_unused_test_images():
    """Find images that weren't used in either training run"""
    damage_files = []
    dirty_files = []
    
    # Read both training file lists to exclude all used images
    used_images = set()
    
    # Original training set
    with open('train.txt', 'r') as f:
        used_images.update(line.strip().split('/')[-1] for line in f)
    with open('val.txt', 'r') as f:
        used_images.update(line.strip().split('/')[-1] for line in f)
    
    # Extended training set
    with open('train_extended.txt', 'r') as f:
        used_images.update(line.strip().split('/')[-1] for line in f)
    with open('val_extended.txt', 'r') as f:
        used_images.update(line.strip().split('/')[-1] for line in f)
    
    print(f"Excluding {len(used_images)} previously used images")
    
    # Categorize unused images
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        img_name = label_file.replace('.txt', '.png')
        
        # Skip if this image was used in any training
        if img_name in used_images:
            continue
            
        if not (images_dir / img_name).exists():
            continue
            
        # Check class
        label_path = labels_dir / label_file
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            dirty_files.append(img_name)
            continue
            
        has_damage = False
        for line in lines:
            if line.strip():
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            damage_files.append(img_name)
        else:
            dirty_files.append(img_name)
    
    return damage_files, dirty_files

# Get completely unused test images
unused_damage, unused_dirty = get_unused_test_images()
print(f"Available unused images: {len(unused_damage)} damage, {len(unused_dirty)} dirty")

# Sample larger test set
test_damage = random.sample(unused_damage, min(100, len(unused_damage)))
test_dirty = random.sample(unused_dirty, min(400, len(unused_dirty)))

print(f"Testing on: {len(test_damage)} damage images, {len(test_dirty)} dirty images")

# Test the model with detailed analysis
def test_damage_sensitive_model(image_list, true_class_name):
    """Test model and analyze confidence distributions"""
    results = []
    high_conf_correct = 0
    low_conf_missed = 0
    
    for img_name in image_list:
        img_path = images_dir / img_name
        
        # Run prediction
        pred_results = model(str(img_path))
        
        # Extract predictions
        detections = []
        if len(pred_results[0].boxes) > 0:
            boxes = pred_results[0].boxes
            for box in boxes:
                class_id = int(box.cls[0].item())
                confidence = float(box.conf[0].item())
                class_name = "damage" if class_id == 0 else "dirty"
                detections.append((class_name, confidence))
        
        # Check if prediction matches true class
        predicted_classes = [det[0] for det in detections]
        has_correct_prediction = true_class_name in predicted_classes
        
        if has_correct_prediction:
            # Check confidence of correct predictions
            correct_confidences = [det[1] for det in detections if det[0] == true_class_name]
            max_conf = max(correct_confidences)
            if max_conf > 0.7:
                high_conf_correct += 1
        else:
            # Missed detection - could be low confidence issue
            if not detections:
                low_conf_missed += 1
        
        results.append({
            'image': img_name,
            'true_class': true_class_name,
            'detections': detections,
            'correct': has_correct_prediction
        })
    
    return results, high_conf_correct, low_conf_missed

# Run comprehensive testing
print("\nTesting damage images...")
damage_results, damage_high_conf, damage_missed = test_damage_sensitive_model(test_damage, 'damage')

print("Testing dirty images...")
dirty_results, dirty_high_conf, dirty_missed = test_damage_sensitive_model(test_dirty, 'dirty')

# Analysis
damage_accuracy = sum(1 for r in damage_results if r['correct']) / len(damage_results)
dirty_accuracy = sum(1 for r in dirty_results if r['correct']) / len(dirty_results)
overall_accuracy = (sum(1 for r in damage_results if r['correct']) + 
                   sum(1 for r in dirty_results if r['correct'])) / (len(damage_results) + len(dirty_results))

print(f"\n=== DAMAGE-SENSITIVE MODEL RESULTS ===")
print(f"Damage detection accuracy: {damage_accuracy:.3f} ({sum(1 for r in damage_results if r['correct'])}/{len(damage_results)})")
print(f"  - High confidence correct: {damage_high_conf}")
print(f"  - No detections (missed): {damage_missed}")

print(f"\nDirty detection accuracy: {dirty_accuracy:.3f} ({sum(1 for r in dirty_results if r['correct'])}/{len(dirty_results)})")
print(f"  - High confidence correct: {dirty_high_conf}")
print(f"  - No detections (missed): {dirty_missed}")

print(f"\nOverall accuracy: {overall_accuracy:.3f}")
print(f"Model saved at: runs/detect/damage_sensitive_finetuned/weights/best.pt")

# Show some examples of missed damage (most critical)
missed_damage = [r for r in damage_results if not r['correct']]
if missed_damage:
    print(f"\n⚠️  CRITICAL: {len(missed_damage)} damage images were missed:")
    for r in missed_damage[:5]:  # Show first 5
        print(f"  {r['image']}: {r['detections'] if r['detections'] else 'No detections'}")

Excluding 1740 previously used images
Available unused images: 219 damage, 1036 dirty
Testing on: 100 damage images, 400 dirty images

Testing damage images...

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0702_04_05.png: 416x640 1 dirty, 154.3ms
Speed: 9.1ms preprocess, 154.3ms inference, 10.1ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0625_02_07.png: 416x640 1 damage, 1 dirty, 72.3ms
Speed: 3.2ms preprocess, 72.3ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0749_04_08.png: 416x640 1 damage, 64.7ms
Speed: 2.8ms preprocess, 64.7ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0757_04_07.png: 416x640

In [3]:
from ultralytics import YOLO

# Load your best existing model
model = YOLO('runs/detect/damage_sensitive_finetuned/weights/best.pt')

# Test with ultra-low threshold for damage detection
def ultra_sensitive_damage_detection(image_list, true_class_name):
    results = []
    
    for img_name in image_list:
        img_path = images_dir / img_name
        
        # Very low confidence threshold, only look for damage class
        pred_results = model.predict(str(img_path), conf=0.02, classes=[0])
        
        # Check if any damage detected
        damage_detected = len(pred_results[0].boxes) > 0
        
        results.append({
            'image': img_name,
            'true_class': true_class_name,
            'damage_detected': damage_detected,
            'correct': damage_detected if true_class_name == 'damage' else not damage_detected
        })
    
    return results

# Test this immediately on your existing test data
damage_results_sensitive = ultra_sensitive_damage_detection(test_damage, 'damage')
dirty_results_sensitive = ultra_sensitive_damage_detection(test_dirty, 'dirty')

damage_recall = sum(1 for r in damage_results_sensitive if r['damage_detected']) / len(damage_results_sensitive)
print(f"Ultra-sensitive damage recall: {damage_recall:.3f}")


image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0702_04_05.png: 416x640 5 damages, 210.5ms
Speed: 7.8ms preprocess, 210.5ms inference, 14.8ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0625_02_07.png: 416x640 1 damage, 172.8ms
Speed: 7.9ms preprocess, 172.8ms inference, 3.2ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0749_04_08.png: 416x640 3 damages, 175.1ms
Speed: 7.5ms preprocess, 175.1ms inference, 3.9ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0757_04_07.png: 416x640 2 damages, 171.8ms
Speed: 6.6ms preprocess, 171.8ms inference, 4.3ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blad

In [4]:
# Analyze the ultra-sensitive test results
total_images = len(damage_results_sensitive) + len(dirty_results_sensitive)
actually_damaged = len(damage_results_sensitive)  # 100
actually_dirty = len(dirty_results_sensitive)    # 400

# Count how many were flagged as damaged by the ultra-sensitive model
flagged_as_damaged = (sum(1 for r in damage_results_sensitive if r['damage_detected']) + 
                      sum(1 for r in dirty_results_sensitive if r['damage_detected']))

print(f"=== ULTRA-SENSITIVE MODEL ANALYSIS ===")
print(f"Total test images: {total_images}")
print(f"Actually damaged: {actually_damaged} ({actually_damaged/total_images:.1%})")
print(f"Actually dirty: {actually_dirty} ({actually_dirty/total_images:.1%})")
print(f"Flagged as damaged: {flagged_as_damaged} ({flagged_as_damaged/total_images:.1%})")

# False positives and false negatives
false_negatives = sum(1 for r in damage_results_sensitive if not r['damage_detected'])
false_positives = sum(1 for r in dirty_results_sensitive if r['damage_detected'])

print(f"\n=== CRITICAL METRICS ===")
print(f"False negatives (missed damage): {false_negatives} ({false_negatives/actually_damaged:.1%})")
print(f"False positives (dirty flagged as damage): {false_positives} ({false_positives/actually_dirty:.1%})")
print(f"Inspection workload reduction: {(total_images - flagged_as_damaged)/total_images:.1%}")

# Show the missed damage cases
missed_damage = [r['image'] for r in damage_results_sensitive if not r['damage_detected']]
print(f"\n⚠️  MISSED DAMAGE IMAGES:")
for img in missed_damage:
    print(f"  {img}")

=== ULTRA-SENSITIVE MODEL ANALYSIS ===
Total test images: 500
Actually damaged: 100 (20.0%)
Actually dirty: 400 (80.0%)
Flagged as damaged: 110 (22.0%)

=== CRITICAL METRICS ===
False negatives (missed damage): 4 (4.0%)
False positives (dirty flagged as damage): 14 (3.5%)
Inspection workload reduction: 78.0%

⚠️  MISSED DAMAGE IMAGES:
  DJI_0680_04_07.png
  DJI_0580_02_04.png
  DJI_0695_03_05.png
  DJI_0687_06_04.png


In [5]:
#New model with highly increased false damage weight

# Load your best existing model as starting point
model = YOLO('runs/detect/damage_sensitive_finetuned/weights/best.pt')

# Train with extreme damage sensitivity
results = model.train(
    data='dataset_extended.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    
    # Extreme loss weighting - much higher than before
    cls=15.0,   # Was 3.0, now 15.0 - massive penalty for misclassification
    box=2.0,    # Slightly higher box loss
    
    # Very conservative training to avoid overfitting
    lr0=0.0005,    # Lower learning rate
    patience=25,    # More patience for early stopping
    
    name='ultra_damage_sensitive',
    
    # Optional: Add more augmentation to make model more robust
    degrees=5.0,    # Slight rotation
    translate=0.1,  # Slight translation
)

print("Training ultra damage-sensitive model...")

Ultralytics 8.3.202  Python-3.12.7 torch-2.8.0+cpu CPU (Intel Core i5-1035G4 1.10GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=2.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=15.0, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_extended.yaml, degrees=5.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/damage_sensitive_finetuned/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ultra_damage_sensitive, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_

In [6]:
# Test the ultra-sensitive model
ultra_model = YOLO('runs/detect/ultra_damage_sensitive/weights/best.pt')

# Run the same test with very low threshold
def test_ultra_model(image_list, true_class_name):
    results = []
    for img_name in image_list:
        img_path = images_dir / img_name
        pred_results = ultra_model.predict(str(img_path), conf=0.01, classes=[0])
        damage_detected = len(pred_results[0].boxes) > 0
        results.append({
            'damage_detected': damage_detected,
            'correct': damage_detected if true_class_name == 'damage' else not damage_detected
        })
    return results

# Test on your same test set
ultra_damage_results = test_ultra_model(test_damage, 'damage')
ultra_dirty_results = test_ultra_model(test_dirty, 'dirty')

damage_recall = sum(1 for r in ultra_damage_results if r['damage_detected']) / len(ultra_damage_results)
print(f"Ultra model damage recall: {damage_recall:.3f}")


image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0702_04_05.png: 416x640 5 damages, 116.3ms
Speed: 8.7ms preprocess, 116.3ms inference, 6.7ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0625_02_07.png: 416x640 1 damage, 64.4ms
Speed: 3.2ms preprocess, 64.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0749_04_08.png: 416x640 1 damage, 57.8ms
Speed: 2.6ms preprocess, 57.8ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damages Images + Labels\NordTank586x371\images\DJI_0757_04_07.png: 416x640 2 damages, 59.7ms
Speed: 2.3ms preprocess, 59.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 C:\Users\dmirk\Sulzer Schmid\Blades Damag

In [8]:
# Analyze the ultra-sensitive test results
total_images = len(test_damage) + len(test_dirty)
actually_damaged = len(test_damage)  # Should be 100
actually_dirty = len(test_dirty)    # Should be 400

# Count how many were flagged as damaged by the ultra-sensitive model
flagged_as_damaged = (sum(1 for r in ultra_damage_results if r['damage_detected']) + 
                      sum(1 for r in ultra_dirty_results if r['damage_detected']))

print(f"=== ULTRA-SENSITIVE MODEL ANALYSIS ===")
print(f"Total test images: {total_images}")
print(f"Actually damaged: {actually_damaged} ({actually_damaged/total_images:.1%})")
print(f"Actually dirty: {actually_dirty} ({actually_dirty/total_images:.1%})")
print(f"Flagged as damaged: {flagged_as_damaged} ({flagged_as_damaged/total_images:.1%})")

# False positives and false negatives
false_negatives = sum(1 for r in ultra_damage_results if not r['damage_detected'])
false_positives = sum(1 for r in ultra_dirty_results if r['damage_detected'])

print(f"\n=== CRITICAL METRICS ===")
print(f"False negatives (missed damage): {false_negatives} ({false_negatives/actually_damaged:.1%})")
print(f"False positives (dirty flagged as damage): {false_positives} ({false_positives/actually_dirty:.1%})")
print(f"Inspection workload reduction: {(total_images - flagged_as_damaged)/total_images:.1%}")

# Show the missed damage cases - corrected to match actual data structure
if false_negatives > 0:
    print(f"\n⚠️  MISSED DAMAGE IMAGES ({false_negatives}):")
    for i, (result, img_name) in enumerate(zip(ultra_damage_results, test_damage)):
        if not result['damage_detected']:
            print(f"  {i+1}. {img_name}")
else:
    print(f"\n✅ NO DAMAGE MISSED!")

print(f"\nModel performance summary:")
print(f"- Started with 93% damage detection")
print(f"- Ultra-sensitive model: 98% damage detection") 
print(f"- Only {false_negatives} out of {actually_damaged} damaged blades missed")
print(f"- Inspection workload: {flagged_as_damaged} images ({flagged_as_damaged/total_images:.1%}) need human inspection")
print(f"- Workload reduction: {(total_images - flagged_as_damaged)/total_images:.1%}")

=== ULTRA-SENSITIVE MODEL ANALYSIS ===
Total test images: 500
Actually damaged: 100 (20.0%)
Actually dirty: 400 (80.0%)
Flagged as damaged: 123 (24.6%)

=== CRITICAL METRICS ===
False negatives (missed damage): 2 (2.0%)
False positives (dirty flagged as damage): 25 (6.2%)
Inspection workload reduction: 75.4%

⚠️  MISSED DAMAGE IMAGES (2):
  7. DJI_0680_04_07.png
  94. DJI_0687_06_04.png

Model performance summary:
- Started with 93% damage detection
- Ultra-sensitive model: 98% damage detection
- Only 2 out of 100 damaged blades missed
- Inspection workload: 123 images (24.6%) need human inspection
- Workload reduction: 75.4%


In [12]:
import os
from pathlib import Path

# Get ALL available images from the dataset
def get_all_dataset_images():
    """Get every image in the dataset with their true labels"""
    all_damage_images = []
    all_dirty_images = []
    
    label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]
    
    for label_file in label_files:
        img_name = label_file.replace('.txt', '.png')
        
        # Check if image exists
        if not (images_dir / img_name).exists():
            continue
            
        # Determine true class
        label_path = labels_dir / label_file
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        if not lines:
            all_dirty_images.append(img_name)
            continue
            
        has_damage = False
        for line in lines:
            if line.strip():
                class_id = int(line.strip().split()[0])
                if class_id == 0:  # Damage class
                    has_damage = True
                    break
        
        if has_damage:
            all_damage_images.append(img_name)
        else:
            all_dirty_images.append(img_name)
    
    return all_damage_images, all_dirty_images

# Get all images in dataset
all_damage_imgs, all_dirty_imgs = get_all_dataset_images()
print(f"Found {len(all_damage_imgs)} damage images, {len(all_dirty_imgs)} dirty images")
print(f"Total dataset: {len(all_damage_imgs) + len(all_dirty_imgs)} images")

# Test on entire dataset with optimal threshold (conf=0.005)
def test_entire_dataset(damage_images, dirty_images):
    """Test ultra-sensitive model on entire dataset"""
    print("\nTesting on entire dataset...")
    
    # Test damage images
    damage_detected = 0
    for i, img_name in enumerate(damage_images):
        if i % 100 == 0:
            print(f"Processing damage images: {i}/{len(damage_images)}")
        
        img_path = images_dir / img_name
        pred_results = ultra_model.predict(str(img_path), conf=0.005, classes=[0], verbose=False)
        if len(pred_results[0].boxes) > 0:
            damage_detected += 1
    
    # Test dirty images  
    dirty_flagged_as_damage = 0
    for i, img_name in enumerate(dirty_images):
        if i % 500 == 0:
            print(f"Processing dirty images: {i}/{len(dirty_images)}")
            
        img_path = images_dir / img_name
        pred_results = ultra_model.predict(str(img_path), conf=0.005, classes=[0], verbose=False)
        if len(pred_results[0].boxes) > 0:
            dirty_flagged_as_damage += 1
    
    return damage_detected, dirty_flagged_as_damage

# Run full dataset test
detected_damage, false_positives = test_entire_dataset(all_damage_imgs, all_dirty_imgs)

# Calculate final metrics
total_images = len(all_damage_imgs) + len(all_dirty_imgs)
total_flagged = detected_damage + false_positives
damage_recall = detected_damage / len(all_damage_imgs)
false_positive_rate = false_positives / len(all_dirty_imgs)

print(f"\n=== FULL DATASET RESULTS ===")
print(f"Total images tested: {total_images}")
print(f"Actually damaged: {len(all_damage_imgs)}")
print(f"Actually dirty: {len(all_dirty_imgs)}")
print(f"")
print(f"Damage recall: {detected_damage}/{len(all_damage_imgs)} = {damage_recall:.1%}")
print(f"Missed damage: {len(all_damage_imgs) - detected_damage}")
print(f"")
print(f"Total flagged for inspection: {total_flagged}/{total_images} = {total_flagged/total_images:.1%}")
print(f"False positives: {false_positives}/{len(all_dirty_imgs)} = {false_positive_rate:.1%}")
print(f"")
print(f"Workload reduction: {(total_images - total_flagged)/total_images:.1%}")
print(f"Human inspectors only need to check {total_flagged} images instead of {total_images}")

Found 563 damage images, 2432 dirty images
Total dataset: 2995 images

Testing on entire dataset...
Processing damage images: 0/563
Processing damage images: 100/563
Processing damage images: 200/563
Processing damage images: 300/563
Processing damage images: 400/563
Processing damage images: 500/563
Processing dirty images: 0/2432
Processing dirty images: 500/2432
Processing dirty images: 1000/2432
Processing dirty images: 1500/2432
Processing dirty images: 2000/2432

=== FULL DATASET RESULTS ===
Total images tested: 2995
Actually damaged: 563
Actually dirty: 2432

Damage recall: 559/563 = 99.3%
Missed damage: 4

Total flagged for inspection: 805/2995 = 26.9%
False positives: 246/2432 = 10.1%

Workload reduction: 73.1%
Human inspectors only need to check 805 images instead of 2995


In [13]:
#Even lower confidence intervall
# Test with conf=0.001 on the 4 missed damage cases
def ultra_low_threshold_test(damage_images, dirty_images):
    """Test with conf=0.001"""
    print("Testing with conf=0.001 on entire dataset...")
    
    # Test damage images
    damage_detected = 0
    for i, img_name in enumerate(damage_images):
        if i % 100 == 0:
            print(f"Processing damage images: {i}/{len(damage_images)}")
        
        img_path = images_dir / img_name
        pred_results = ultra_model.predict(str(img_path), conf=0.001, classes=[0], verbose=False)
        if len(pred_results[0].boxes) > 0:
            damage_detected += 1
    
    # Test dirty images  
    dirty_flagged_as_damage = 0
    for i, img_name in enumerate(dirty_images):
        if i % 500 == 0:
            print(f"Processing dirty images: {i}/{len(dirty_images)}")
            
        img_path = images_dir / img_name
        pred_results = ultra_model.predict(str(img_path), conf=0.001, classes=[0], verbose=False)
        if len(pred_results[0].boxes) > 0:
            dirty_flagged_as_damage += 1
    
    return damage_detected, dirty_flagged_as_damage

# Test with ultra-low threshold
detected_damage_001, false_positives_001 = ultra_low_threshold_test(all_damage_imgs, all_dirty_imgs)

total_flagged_001 = detected_damage_001 + false_positives_001
damage_recall_001 = detected_damage_001 / len(all_damage_imgs)
false_positive_rate_001 = false_positives_001 / len(all_dirty_imgs)

print(f"\n=== ULTRA-LOW THRESHOLD (conf=0.001) RESULTS ===")
print(f"Damage recall: {detected_damage_001}/{len(all_damage_imgs)} = {damage_recall_001:.1%}")
print(f"Missed damage: {len(all_damage_imgs) - detected_damage_001}")
print(f"")
print(f"Total flagged for inspection: {total_flagged_001}/{total_images} = {total_flagged_001/total_images:.1%}")
print(f"False positives: {false_positives_001}/{len(all_dirty_imgs)} = {false_positive_rate_001:.1%}")
print(f"Workload reduction: {(total_images - total_flagged_001)/total_images:.1%}")

print(f"\n=== COMPARISON ===")
print(f"conf=0.005: {559/563:.1%} recall, 805 flagged ({805/2995:.1%})")
print(f"conf=0.001: {damage_recall_001:.1%} recall, {total_flagged_001} flagged ({total_flagged_001/total_images:.1%})")
print(f"Trade-off: +{total_flagged_001-805} more inspections to catch {detected_damage_001-559} more damage cases")

Testing with conf=0.001 on entire dataset...
Processing damage images: 0/563
Processing damage images: 100/563
Processing damage images: 200/563
Processing damage images: 300/563
Processing damage images: 400/563
Processing damage images: 500/563
Processing dirty images: 0/2432
Processing dirty images: 500/2432
Processing dirty images: 1000/2432
Processing dirty images: 1500/2432
Processing dirty images: 2000/2432

=== ULTRA-LOW THRESHOLD (conf=0.001) RESULTS ===
Damage recall: 563/563 = 100.0%
Missed damage: 0

Total flagged for inspection: 1357/2995 = 45.3%
False positives: 794/2432 = 32.6%
Workload reduction: 54.7%

=== COMPARISON ===
conf=0.005: 99.3% recall, 805 flagged (26.9%)
conf=0.001: 100.0% recall, 1357 flagged (45.3%)
Trade-off: +552 more inspections to catch 4 more damage cases
